In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.|

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from nltk.corpus import stopwords

# some linear models
from sklearn.linear_model import LogisticRegression, BayesianRidge

# SCM for classification
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')
np.random.seed(2019)

In [3]:
# run once to download data
# import nltk
# nltk.download()

In [4]:
from sklearn.metrics import accuracy_score

In [5]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_cat=pd.read_json('categories.json')

In [6]:
df_train['Group']=df_train.image_path.map(lambda x: x[:7])
df_test['Group']=df_test.image_path.map(lambda x: x[:7])

In [7]:
train,test=train_test_split(df_train,random_state=2019,stratify=df_train.Category,test_size=0.2)

In [8]:
# Let's check few examples
for i in range(10):
    print(train.title.iloc[i])

btb new blus casual wanita tanpa lengan dengan bahan sifon dan potongan longgar bergaya sexy
iphone 7 plus 128gb red fullset original
sale ben nye neutral set colorless powder poudre incolore 25 g
terlaris dermovate cream hijau original
termurah mini dress brokat gaun pesta murah party wanita
samasung s7 flat
obral 2018 new cotton checkered plaid blouses shirt cage female long sleeve casual slim women
marcks bedak tabur 20 g new cn86
new loreal paris true match blur cream 25ml stok terbatas
sweater kaos hoodie lengan panjang bahan velvet untuk pria wanita


## Building seperate models for each Groupb

In [12]:
ignore_words = set(stopwords.words('english')).union(set(stopwords.words('indonesian')))
print(len(ignore_words))

936


In [10]:
# let's check for all of the data. This slightly helps to reduce dimensionality
cv=CountVectorizer(stop_words=ignore_words)
cv1=CountVectorizer() # No, stop words
X_t=cv.fit_transform(train['title'])
X_t1=cv1.fit_transform(train['title'])
print('Some words are reduced by introducing stop words',X_t.shape[1],X_t1.shape[1])

Some words are reduced by introducing stop words 70586 70968


In [16]:
# let's build pipeline for parameter search in n_grams and and various models
lr=LogisticRegression()
nb=MultinomialNB()
svc=SVC()
# let's check first three and see which one is best
gnb=GaussianNB()
bnb=BernoulliNB()
br=BayesianRidge()

pipe_lr=Pipeline([('vectorize', CountVectorizer(stop_words =ignore_words)),('model',lr)])


In [12]:
%time pipe_lr.fit(train[train.Group=='beauty_'].title,train[train.Group=='beauty_'].Category)

Wall time: 1min 20s


Pipeline(memory=None,
     steps=[('vectorize', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words={'jang...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [14]:
%%time
pred=pipe_lr.predict(test[test.Group=='beauty_'].title)

Wall time: 416 ms


In [15]:
%%time
accuracy_score(test[test.Group=='beauty_'].Category,pred)


# without removing stop words :0.76361288971858265
# removing stop_words: 0.7626009735331577

Wall time: 32.9 ms


0.7626009735331577

In [16]:
%%time
score=pipe_lr.score(test[test.Group=='beauty_'].title,test[test.Group=='beauty_'].Category)

Wall time: 446 ms


In [9]:
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

## Previously I have tried both TfidfVectorizer, CountVectorizer for both naive bayes and logistic regression. But it turns out that Countvectorizer with logistic regression gives best results. I have removed stop words from english and indonesian. Than, I did parameter optimization for n_grams. It turns out that best ngram_range is (1,3 for beauty and mobile products and (1,5) for fashion. 

## (1,5) might be not the most optimal we might still imporove by increasing it.


## I also run this simulation for NB. The optimal ngram_range was ... I lost it in kernel!!

## but it was smth like for mobile (1,2) and for beauty and fashion (2,5) we can do it again, it won't take much time. Score were still less than logistic regression

## I also tried svc (SVM) but it takes to much time imposimble to train espacially if there is many features



## Let's predict with naive bayes approach

In [17]:
pipe_nb_beauty=Pipeline([('vectorize', CountVectorizer(stop_words =ignore_words,ngram_range=(2,6))),('model',nb)])
pipe_nb_mobile=Pipeline([('vectorize', CountVectorizer(stop_words =ignore_words,ngram_range=(1,2))),('model',nb)])
pipe_nb_fashion=Pipeline([('vectorize', CountVectorizer(stop_words =ignore_words,ngram_range=(2,6))),('model',nb)])

pipe_nb_all=[pipe_nb_beauty,pipe_nb_mobile,pipe_nb_fashion]


In [18]:
groups

['beauty_', 'mobile_', 'fashion']

In [19]:
# Let's get predictions and predict proba for naive bayes approch
pred_nb_proba_val={}
pred_nb_proba_test={}
classes={}
classes_all={}
for i in range(3):
    model=pipe_nb_all[i]
    X=train[train.Group==groups[i]].title
    y=train[train.Group==groups[i]].Category
    model.fit(X,y)
    classes[groups[i]]=model.classes_
    pred_nb_proba_val[groups[i]]=model.predict_proba(test[test.Group==groups[i]].title)
    # Train to all data
    model=pipe_nb_all[i]
    X=df_train[df_train.Group==groups[i]].title
    y=df_train[df_train.Group==groups[i]].Category
    model.fit(X,y)
    classes_all[groups[i]]=model.classes_
    pred_nb_proba_test[groups[i]]=model.predict_proba(df_test[df_test.Group==groups[i]].title)

In [20]:
# Saving probabilities of validation set
for i in range(3):
    df=pd.DataFrame(pred_nb_proba_val[groups[i]])
    df.columns=classes[groups[i]]
    df['pred']=df.idxmax(axis=1) # do it before placing itemid or category otherwise column type will become string type
    df['itemid']=test[test.Group==groups[i]].itemid.values # values is essensial since indexes are different
    df['Category']=test[test.Group==groups[i]].Category.values
    df.to_csv(groups[i]+'_val_proba_nlp_nb.csv',index=False)
    

In [21]:
# Saving Predictions for final test set
for i in range(3):
    df=pd.DataFrame(pred_nb_proba_test[groups[i]])
    df.columns=classes[groups[i]]
    df['pred']=df.idxmax(axis=1)
    df['itemid']=df_test[df_test.Group==groups[i]].itemid.values
    df.to_csv(groups[i]+'_test_proba_nlp_nb.csv',index=False)

# Train for the whole model and save test predict proba

In [57]:
# Let's now use all data and save predict proba of model with all data
pred_nb_proba_test_all={}
classes={}
for i in range(3):
    model=pipe_nb_all[i]
    X=df_train[df_train.Group==groups[i]].title
    y=df_train[df_train.Group==groups[i]].Category
    model.fit(X,y)
    classes[groups[i]]=model.classes_
    pred_nb_proba_test_all[groups[i]]=model.predict_proba(df_test[df_test.Group==groups[i]].title)

In [58]:
# Saving Predictions for final test set
for i in range(3):
    df=pd.DataFrame(pred_nb_proba_test_all[groups[i]])
    df.columns=classes[groups[i]]
    df['pred']=df.idxmax(axis=1)
    df['itemid']=df_test[df_test.Group==groups[i]].itemid.values
    df.to_csv(groups[i]+'_test_proba_nlp_nb_all.csv',index=False)